In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

df = spark.read.option("header", "true").csv("data/steam_reviews.csv")
df.show()



24/02/04 21:09:06 WARN Utils: Your hostname, Alexanders-MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 192.168.2.6 instead (on interface en0)
24/02/04 21:09:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 21:09:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------------------------+----------+--------------------+---------+--------+------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|                        _c0|    app_id|            app_name|review_id|language|                              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+---------------------------+----------+--------------------+---------+--------+--------------------------

24/02/04 21:09:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: file:///Users/alexandergunawan/data_engineer/data/steam_reviews.csv


In [2]:
df.schema

StructType([StructField('_c0', StringType(), True), StructField('app_id', StringType(), True), StructField('app_name', StringType(), True), StructField('review_id', StringType(), True), StructField('language', StringType(), True), StructField('review', StringType(), True), StructField('timestamp_created', StringType(), True), StructField('timestamp_updated', StringType(), True), StructField('recommended', StringType(), True), StructField('votes_helpful', StringType(), True), StructField('votes_funny', StringType(), True), StructField('weighted_vote_score', StringType(), True), StructField('comment_count', StringType(), True), StructField('steam_purchase', StringType(), True), StructField('received_for_free', StringType(), True), StructField('written_during_early_access', StringType(), True), StructField('author.steamid', StringType(), True), StructField('author.num_games_owned', StringType(), True), StructField('author.num_reviews', StringType(), True), StructField('author.playtime_for

In [4]:
# we reaplce all the dots with _ in the column names
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

new_columns = [c.replace(".", "_") for c in df.columns]

df = df.toDF(*new_columns)
df.schema


StructType([StructField('_c0', StringType(), True), StructField('app_id', StringType(), True), StructField('app_name', StringType(), True), StructField('review_id', StringType(), True), StructField('language', StringType(), True), StructField('review', StringType(), True), StructField('timestamp_created', StringType(), True), StructField('timestamp_updated', StringType(), True), StructField('recommended', StringType(), True), StructField('votes_helpful', StringType(), True), StructField('votes_funny', StringType(), True), StructField('weighted_vote_score', StringType(), True), StructField('comment_count', StringType(), True), StructField('steam_purchase', StringType(), True), StructField('received_for_free', StringType(), True), StructField('written_during_early_access', StringType(), True), StructField('author_steamid', StringType(), True), StructField('author_num_games_owned', StringType(), True), StructField('author_num_reviews', StringType(), True), StructField('author_playtime_for

In [5]:
#change the author.playtime_forever to int:
from pyspark.sql.functions import round
df = df.withColumn("author_playtime_forever_hrs", round(df["author_playtime_forever"].cast(IntegerType())/60))



In [21]:
df.withColumn('numNulls', sum(df[col].isNull().cast('int') for col in df.columns)).show()
#seems like thre are a few rowstaht are weird, as in all null. we will drop these rows.


+---------------------------+----------+--------------------+---------+--------+------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------------------------+--------+
|                        _c0|    app_id|            app_name|review_id|language|                              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|author_playtime_forever_hrs|numNulls|
+---------------------------+---

24/02/04 21:40:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: file:///Users/alexandergunawan/data_engineer/data/steam_reviews.csv


In [24]:
df = df.na.drop() #drop rows


In [25]:
# figure out how many hours are played - the top 1% percentile. 
# This is a good way to find out how many hours are played by the most active players

df.createOrReplaceTempView("steam_reviews") #created temporary view named steam_reviews using df.
result = spark.sql("SELECT percentile_approx(author_playtime_forever_hrs, 0.95) FROM steam_reviews")
result.show(truncate=False)

# the 95th percentile played more than 1436 hours. 

24/02/04 21:43:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: file:///Users/alexandergunawan/data_engineer/data/steam_reviews.csv


+-----------------------------------------------------------+
|percentile_approx(author_playtime_forever_hrs, 0.95, 10000)|
+-----------------------------------------------------------+
|1193.0                                                     |
+-----------------------------------------------------------+



In [26]:
#create a new table app_id_table that contains the app_id and the title of the game
spark.sql("SELECT DISTINCT app_id, app_name FROM steam_reviews").show(5)

24/02/04 21:51:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: file:///Users/alexandergunawan/data_engineer/data/steam_reviews.csv


In [68]:
\\

24/02/01 22:38:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: file:///Users/alexandergunawan/data_engineer/data/steam_reviews.csv


+---+------+--------+---------+--------+------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+--------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------------------------+
|_c0|app_id|app_name|review_id|language|review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|author_playtime_forever_hrs|
+---+------+--------+---------+--------+------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+--------

In [65]:
\

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[app_id#2043, app_name#2044], functions=[])
   +- Exchange hashpartitioning(app_id#2043, app_name#2044, 200), ENSURE_REQUIREMENTS, [plan_id=650]
      +- HashAggregate(keys=[app_id#2043, app_name#2044], functions=[])
         +- FileScan csv [app_id#2043,app_name#2044] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/alexandergunawan/data_engineer/data/steam_reviews.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<app_id:string,app_name:string>


